Usage
========

First, some imports

In [6]:
import rustworkx as rx

from territories import Part, Territory, Partition

## Creation of the tree

The first step is to create a tree of known entities. This can be a very compute intensive task, depending on the tree size.

Here we'll create a very simple tree.

In [7]:

lyon = Part("Lyon")
marseille = Part("Marseille", es_code="COM:2909") # you can specify an ElasticSearch code
paris = Part("Paris")
nogent = Part("Nogent")
pantin = Part("Pantin")
villeurbane = Part("Villeurbane")
sté = Part("Saint Etienne")

metropole = Part("Grand Lyon", False, Partition.DEP)

sud = Part("Sud", False, Partition.REGION)
idf = Part("Île-de-France", False, Partition.REGION)
rhone = Part("Rhône", False, Partition.DEP)

france = Part("France", False, Partition.COUNTRY)


# buiding the reference Part tree
entities = (france, sud, idf, rhone, metropole, nogent, pantin, paris, marseille, sté, villeurbane, lyon)

tree = rx.PyDiGraph()
entities_indices = tree.add_nodes_from(entities)

mapper = {o : idx for o, idx in zip(entities, entities_indices)}
edges = [
    (france, idf),
    (france, sud),
    
    (idf, nogent),
    (idf, pantin),
    (idf, paris),

    (sud, marseille),
    (sud, rhone),

    (rhone, metropole),
    (rhone, sté),

    (metropole, villeurbane),
    (metropole, lyon),
    ]

tree.add_edges_from([
    (mapper[parent], mapper[child], None) for parent, child in edges
])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Once the tree is created, you need to assign it.

In [8]:
Territory.assign_tree(tree)

Then, you can start to create territories from arbitrary entities.

In [9]:
a = Territory(sté, marseille)
b = Territory(lyon, france)
c = Territory(paris, nogent, pantin, lyon, lyon, metropole)
d = Territory(lyon, villeurbane, marseille)
e = Territory(rhone, idf)
f = Territory(idf, marseille, metropole)

Entities associated territories are represented in an efficient way : if all leaves of a parent node are included in the territory, they are simply replaced by their parent node.

In [10]:
b # since Lyon is included in France, only France remains

France

## Operations on entities

Usual operation on territories works as expected :

In [11]:
# addition

print(a, c)
print(a + c) # This simplify to France

Saint Etienne|Marseille Île-de-France|Grand Lyon
France


In [12]:
# substraction

print(a, d)
print(a - d) # only Saint Etienne remains

Saint Etienne|Marseille Marseille|Grand Lyon
Saint Etienne


More importantly, sets operations are also supported

In [13]:
# intersection
print(f"Intersection of {a} and {d} is {a & d}")

# union
print(f"Union of {c} and {f} is {f | c}")

Intersection of Saint Etienne|Marseille and Marseille|Grand Lyon is Marseille
Union of Île-de-France|Grand Lyon and Île-de-France|Marseille|Grand Lyon is Île-de-France|Marseille|Grand Lyon
